In [1]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

# from sklearn.linear_model import LinearRegression ## likes, dislikes 대체
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

import os
from dotenv import load_dotenv

In [2]:
file_path = 'C:/py_src/awake/data/'

In [3]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv(file_path + 'merge_df_users_fin_eda.csv', low_memory=False)
# youtube_videos = pd.read_csv(file_path + 'youtube_videos_eda.csv')

## 계정 데이터 분석

In [4]:
# 불필요정보 제거 - y값 제거(다른 모델)
merge_df_users_fin = merge_df_users_fin.drop('y_label',axis=1)
## y값 : merge_df_users_fin['subscribers_count']

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [5]:
# 컬럼 정리
unique_col = merge_df_users_fin.columns[:11]
x_col = merge_df_users_fin.columns[11:].drop('subscribers_count') ## y값 제거

In [6]:
# 데이터 분할
train_data = merge_df_users_fin[merge_df_users_fin['date'] <= '2024-02-11']
test_data = merge_df_users_fin[merge_df_users_fin['date'] > '2024-02-11']

In [7]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(84473, 88)
(21210, 88)


### 변수선택

상관분석

In [8]:
# 상관계수 절대값이 0.3 이상
corr_df = train_data[['subscribers_count'] + list(x_col)].corr()
selected_features_by_corr = list(corr_df[corr_df['subscribers_count'] >= 0.3].iloc[:,0].keys()) + list(corr_df[corr_df['subscribers_count'] <= -0.3].iloc[:,0].keys())
selected_features_by_corr.remove('subscribers_count')
print(selected_features_by_corr)
print(len(selected_features_by_corr))

Lasso

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data[x_col])

# LassoCV 모델 설정: 반복 횟수 증가, alpha 범위 조정
lasso = LassoCV(cv=5, random_state=42, max_iter=50000, alphas=[0.1, 0.05, 0.01, 0.005, 0.001], selection='random').fit(X_train_scaled, train_data['subscribers_count'])

# 선택된 변수 확인 (회귀 계수가 0이 아닌 변수들)
selected_features_by_lasso = x_col.columns[(lasso.coef_ != 0)]
print("Selected Features by Lasso:", selected_features_by_lasso)

c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 259427014331391.6, tolerance: 206333202141.6521
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 264135452986404.75, tolerance: 206333202141.6521
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 304181140066726.2, tolerance: 206333202141.6521
  model = cd_fast.enet_coordinate_descent_gram(
c:\Use

In [39]:
len(selected_features_by_lasso)

45

In [40]:
selected_features_by_lasso

Index(['redViews', 'SUBSCRIBED', 'comments', 'likes', 'shares',
       'estimatedMinutesWatched', 'averageViewDuration', 'estimated_revenue',
       'estimated_red_partner_revenue', 'gross_revenue', 'cpm',
       'subscribers_count', 'subscribers_gained', 'subscribers_lost',
       'ad_impressions', 'monetized_playbacks', 'age13-17.female',
       'age18-24.female', 'age18-24.male', 'age25-34.female', 'age25-34.male',
       'age35-44.female', 'age65-.female', 'like_rate', 'comment_rate',
       'dislike_rate', 'positive_engage_rate', 'like_to_dislike_ratio',
       'subscriber_increase_rate', 'subscriber_decrease_rate',
       'subscribers_conversion_rate', 'subscriber_retention_rate',
       'subscribed_view_rate', 'unsubscribed_view_rate',
       'revenue_per_subscriber', 'revenue_per_red_view', 'ad_revenue_rate',
       'red_revenue_rate', 'revenue_per_minute_watched',
       'avg_view_duration_rate', 'watched_view_rate',
       'subscribed_view_time_rate', 'unsubscribed_view_time_

In [42]:
## lasso 선택으로 제거된 변수
set(selected_features_by_ttest) - set(selected_features_by_lasso)

{'UNSUBSCRIBED',
 'adult_viewer_rate',
 'age13-17.male',
 'age35-44.male',
 'age45-54.female',
 'age55-64.female',
 'age55-64.male',
 'age65-.male',
 'cpm_to_revenue_ratio',
 'dislikes',
 'estimated_ad_revenue',
 'female_viewer_rate',
 'male_viewer_rate',
 'older_viewer_rate',
 'playback_based_cpm',
 'revenue_per_ad_impression',
 'revenue_per_playback',
 'revenue_per_subscribed_view',
 'revenue_per_unsubscribed_view',
 'share_rate',
 'unplayback_rate',
 'views',
 'watched_time_rate',
 'youth_viewer_rate'}

RandomForest

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# RandomForest 모델 학습 및 교차 검증
X_train_lasso_selected = X_train_resampled[selected_features_by_lasso]

model = RandomForestClassifier(random_state=42)
cross_val_scores = cross_val_score(model, X_train_lasso_selected, y_train_resampled, cv=5, scoring='accuracy')

# 모델 학습
model.fit(X_train_lasso_selected, y_train_resampled)

# 피처 중요도 추출
importances = model.feature_importances_
feature_importances = pd.DataFrame({'feature': selected_features_by_lasso, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# 최종적으로 중요한 변수 선택 (예: 상위 10개 변수)
final_selected_features = list(feature_importances[feature_importances['importance'] > 0.005]['feature'])
print("Final Selected Features by RandomForest:", final_selected_features)

# 교차 검증 결과 출력
print(f"Cross-Validation Accuracy: {cross_val_scores.mean():.2f}")

Final Selected Features by RandomForest: ['subscribers_gained', 'likes', 'estimatedMinutesWatched', 'estimated_revenue', 'revenue_per_red_view', 'positive_engage_rate', 'estimated_red_partner_revenue', 'revenue_per_minute_watched', 'comments', 'shares', 'like_rate', 'monetized_playbacks', 'gross_revenue', 'revenue_per_subscriber', 'subscribers_lost', 'redViews', 'cpm', 'subscriber_increase_rate', 'ad_revenue_rate', 'playback_rate', 'subscriber_view_time_rate', 'red_revenue_rate', 'watched_view_rate', 'subscribers_conversion_rate', 'avg_view_duration_rate', 'averageViewDuration', 'comment_rate', 'subscriber_decrease_rate', 'SUBSCRIBED', 'subscribers_count', 'unsubscribed_view_time_rate', 'subscriber_retention_rate', 'like_to_dislike_ratio', 'ad_impressions', 'subscribed_view_rate']
Cross-Validation Accuracy: 0.98


In [47]:
final_selected_features

['subscribers_gained',
 'likes',
 'estimatedMinutesWatched',
 'estimated_revenue',
 'revenue_per_red_view',
 'positive_engage_rate',
 'estimated_red_partner_revenue',
 'revenue_per_minute_watched',
 'comments',
 'shares',
 'like_rate',
 'monetized_playbacks',
 'gross_revenue',
 'revenue_per_subscriber',
 'subscribers_lost',
 'redViews',
 'cpm',
 'subscriber_increase_rate',
 'ad_revenue_rate',
 'playback_rate',
 'subscriber_view_time_rate',
 'red_revenue_rate',
 'watched_view_rate',
 'subscribers_conversion_rate',
 'avg_view_duration_rate',
 'averageViewDuration',
 'comment_rate',
 'subscriber_decrease_rate',
 'SUBSCRIBED',
 'subscribers_count',
 'unsubscribed_view_time_rate',
 'subscriber_retention_rate',
 'like_to_dislike_ratio',
 'ad_impressions',
 'subscribed_view_rate']

In [57]:
feature_importances.iloc[:25]

,feature,importance
12,subscribers_gained,0.099789
3,likes,0.089748
5,estimatedMinutesWatched,0.076199
7,estimated_revenue,0.075472
35,revenue_per_red_view,0.067789
26,positive_engage_rate,0.055985
8,estimated_red_partner_revenue,0.047307
38,revenue_per_minute_watched,0.047070
2,comments,0.040712
4,shares,0.031964


In [44]:
feature_importances ## 0.005 기준으로 변수 선택

,feature,importance
12,subscribers_gained,0.099789
3,likes,0.089748
5,estimatedMinutesWatched,0.076199
7,estimated_revenue,0.075472
35,revenue_per_red_view,0.067789
26,positive_engage_rate,0.055985
8,estimated_red_partner_revenue,0.047307
38,revenue_per_minute_watched,0.047070
2,comments,0.040712
4,shares,0.031964


In [46]:
len(final_selected_features)

35

In [66]:
# 최종 모델링 데이터셋
merge_df_users_final = merge_df_users_fin[list(unique_col)+final_selected_features+['y_label']]

In [67]:
# merge_df_users_final.to_csv('C:/py_src/awake/data/merge_df_users_final.csv', encoding='utf-8-sig', index=False)

## 콘텐츠 데이터 분석

### 데이터 분할

In [73]:
# 컬럼 정리
unique_col = youtube_videos.columns[:3]
x_col = youtube_videos.columns[3:-1]

In [77]:
# 데이터 분할
X = youtube_videos[x_col].drop(columns=['total_engage_rate', 'net_subscribers_change', 'averageViewPercentage', 'revenue_per_view', 'grossRevenue_per_ad_impression','total_card_teaser_click_rate','playlist_engagement_rate']) ## y값 라벨링에 쓰인 지표 제거
y = youtube_videos['y_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# del X, y

In [78]:
# 불균형 확인
print(y_train.value_counts())
print(y_test.value_counts())

y_label
1    6513105
0     342503
Name: count, dtype: int64
y_label
1    1627933
0      85970
Name: count, dtype: int64


### 언더샘플링

In [79]:
# 언더샘플링
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

### 변수선택

t-test

In [80]:
from scipy.stats import ttest_ind

# y=0과 y=1 그룹으로 데이터 나누기
group_0 = X_train_resampled[y_train_resampled == 0]
group_1 = X_train_resampled[y_train_resampled == 1]

# t-test 수행
p_values = []
for col in X_train_resampled.columns:
    t_stat, p_val = ttest_ind(group_0[col], group_1[col], equal_var=False)  # Welch's t-test
    p_values.append((col, p_val))

# p-value < 0.05인 변수 선택
selected_features_by_ttest = [col for col, p_val in p_values if p_val < 0.05]
print("Selected Features by t-test:", selected_features_by_ttest)

Selected Features by t-test: ['views', 'redViews', 'comments', 'likes', 'dislikes', 'shares', 'estimatedMinutesWatched', 'estimatedRedMinutesWatched', 'averageViewDuration', 'videosAddedToPlaylists', 'videosRemovedFromPlaylists', 'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue', 'estimatedRedPartnerRevenue', 'playbackBasedCpm', 'cpm', 'subscribersGained', 'subscribersLost', 'monetizedPlaybacks', 'adImpressions', 'cardClickRate', 'cardTeaserClickRate', 'cardImpressions', 'cardTeaserImpressions', 'cardClicks', 'cardTeaserClicks', 'like_rate', 'comment_rate', 'share_rate', 'dislike_rate', 'positive_engage_rate', 'comment_to_like_rate', 'like_to_dislike_ratio', 'subscribers_conversion_rate', 'subscribers_gained_per_card_click', 'subscribers_gained_per_playlist_add', 'card_click_to_subscriber_conversion', 'subscribers_lost_per_playlist_remove', 'revenue_per_red_view', 'ad_revenue_rate', 'red_revenue_rate', 'cpm_to_revenue_ratio', 'revenue_per_ad_impression', 'playback_based_cpm_rate

In [81]:
len(selected_features_by_ttest)

72

Lasso

In [86]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# t-test로 선택된 변수로 데이터프레임 구성
X_train_ttest_selected = X_train_resampled[selected_features_by_ttest]

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ttest_selected)

# LassoCV 모델 설정: 반복 횟수 증가, alpha 범위 조정
lasso = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=[0.1, 0.05, 0.01, 0.005, 0.001]).fit(X_train_scaled, y_train_resampled)

# 선택된 변수 확인 (회귀 계수가 0이 아닌 변수들)
selected_features_by_lasso = X_train_ttest_selected.columns[(lasso.coef_ != 0)]
print("Selected Features by Lasso:", selected_features_by_lasso)

Selected Features by Lasso: Index(['views', 'redViews', 'comments', 'likes', 'dislikes', 'shares',
       'estimatedRedMinutesWatched', 'averageViewDuration',
       'videosAddedToPlaylists', 'videosRemovedFromPlaylists',
       'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue',
       'playbackBasedCpm', 'cpm', 'subscribersGained', 'subscribersLost',
       'monetizedPlaybacks', 'adImpressions', 'cardClickRate',
       'cardTeaserClickRate', 'cardImpressions', 'cardTeaserImpressions',
       'like_rate', 'comment_rate', 'dislike_rate', 'positive_engage_rate',
       'comment_to_like_rate', 'like_to_dislike_ratio',
       'subscribers_conversion_rate', 'subscribers_gained_per_playlist_add',
       'revenue_per_red_view', 'ad_revenue_rate', 'red_revenue_rate',
       'cpm_to_revenue_ratio', 'revenue_per_ad_impression',
       'playback_based_cpm_rate', 'revenue_per_playlist_add',
       'net_revenue_per_playlist_add', 'revenue_per_minute_watched',
       'avg_view_duration_rate',

In [87]:
len(selected_features_by_lasso)

54

In [91]:
## lasso 선택으로 제거된 변수
set(selected_features_by_ttest) - set(selected_features_by_lasso)

{'ad_impressions_per_card_click',
 'ad_impressions_per_playlist_add',
 'ad_playbacks_per_card_click',
 'ad_revenue_per_card_click',
 'cardClicks',
 'cardTeaserClicks',
 'card_click_to_subscriber_conversion',
 'card_to_teaser_click_rate',
 'estimatedMinutesWatched',
 'estimatedRedPartnerRevenue',
 'net_playlist_addition_rate',
 'playback_rate',
 'revenue_per_card_click',
 'share_rate',
 'subscribers_gained_per_card_click',
 'subscribers_lost_per_playlist_remove',
 'watch_time_per_card_click',
 'watched_time_rate'}

RandomForest

In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# RandomForest 모델 학습 및 교차 검증
X_train_lasso_selected = X_train_resampled[selected_features_by_lasso]

model = RandomForestClassifier(random_state=42)
cross_val_scores = cross_val_score(model, X_train_lasso_selected, y_train_resampled, cv=5, scoring='accuracy')

# 모델 학습
model.fit(X_train_lasso_selected, y_train_resampled)

# 피처 중요도 추출
importances = model.feature_importances_
feature_importances = pd.DataFrame({'feature': selected_features_by_lasso, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# 최종적으로 중요한 변수 선택 (예: 상위 10개 변수)
final_selected_features = list(feature_importances[feature_importances['importance'] > 0.005]['feature'])
print("Final Selected Features by RandomForest:", final_selected_features)

# 교차 검증 결과 출력
print(f"Cross-Validation Accuracy: {cross_val_scores.mean():.2f}")

Final Selected Features by RandomForest: ['positive_engage_rate', 'playlist_addition_rate', 'cpm', 'revenue_per_ad_impression', 'like_rate', 'revenue_per_playback', 'playlist_removal_rate', 'videosAddedToPlaylists', 'videosRemovedFromPlaylists', 'views', 'likes', 'subscribersGained', 'watch_time_per_playlist_add', 'playbackBasedCpm', 'revenue_per_red_view', 'averageViewDuration', 'shares', 'estimatedRevenue', 'avg_view_duration_rate', 'redViews', 'watch_time_loss_per_playlist_remove', 'ad_revenue_rate', 'playlist_related_revenue_rate', 'estimatedRedMinutesWatched', 'watched_view_rate', 'grossRevenue', 'revenue_per_minute_watched', 'comment_rate', 'net_revenue_per_playlist_add', 'cpm_to_revenue_ratio', 'ad_playbacks_per_playlist_add', 'adImpressions', 'estimatedAdRevenue', 'comments', 'subscribers_conversion_rate', 'monetizedPlaybacks', 'dislikes']
Cross-Validation Accuracy: 0.99


In [102]:
len(final_selected_features)

37

In [94]:
feature_importances ## 0.005 기준으로 변수 선택

,feature,importance
26,positive_engage_rate,0.111536
51,playlist_addition_rate,0.079711
14,cpm,0.077948
35,revenue_per_ad_impression,0.064829
23,like_rate,0.064504
45,revenue_per_playback,0.054518
52,playlist_removal_rate,0.045625
8,videosAddedToPlaylists,0.038797
9,videosRemovedFromPlaylists,0.034812
0,views,0.033525


In [100]:
youtube_videos_final = youtube_videos[list(unique_col) + final_selected_features + ['y_label']]

In [101]:
# youtube_videos_final.to_csv('C:/py_src/awake/data/youtube_videos_final.csv', encoding='utf-8-sig', index=False)